# Introduction to OpenTelemetry

OpenTelemetry is a collection of tools, APIs and SDKs that allows for instrumentation of applications to generate observability data that can then be collected and contextualized to help analyze application performance and behavior.

### Relevant OpenTelemetry Terminology

    * API - In the OpenTelemetry project, API's are used to define how telemetry data is generated
    
    * SDK - A telemetry SDK implements the OpenTelemetry API in a specific language
    
    * Semantic Conventions - Defines standard names and values of Metadata in order to provide vendor-agnostic telemetry data
    
    * OTLP - OpenTelemetry Protocol (OTLP) defines the protocol used to exchange data between the client and the server over gRPC or HTTP

### OpenTelemetry Signal Types
    
    * Metrics - a set of measurements collected at regular intervals
    
    * Logs - strings of structured or unstructured text with an associated timestamp
    
    * Traces - chains of events (or transactions) between different components in an application

### OpenTelemetry reference Architecture

Following is a high level architecture diagram for OpenTelemetry. Basically, it shows instrumentation of modern (or legacy) applications that ship back the performance data to a centralized OpenTelemetry Collector: 

![alt text]https://github.com/prathjan/images/blob/main/otelref.png?raw=true

Various pipelines in the collector can be leveraged to contextualize the data before fanning out the observability data to backends such as Jaeger, Prometheus, AppDynamics Cloud or Cisco's FSO Platform:

![alt text]https://github.com/prathjan/images/blob/main/otelref2.png?raw=true

Let's now deploy and auto instrument a sample JAVA application on a k8s cluster, collect/forward observability data to an OTEL collector and then export to backend observability tools, such as **Cisco's Full Stack Observability Platform**. 


## Pre-execution steps

Recall the globals set in the previous notebook. Click Run to set the environment.

In [ ]:
%pwd
%cd output
%ls
!terraform init
!terraform apply -auto-approve
%cd ..


In [ ]:
%env cluster_ca=LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSURCVENDQWUyZ0F3SUJBZ0lJVU5MYmM2SXZ1TGd3RFFZSktvWklodmNOQVFFTEJRQXdGVEVUTUJFR0ExVUUKQXhNS2EzVmlaWEp1WlhSbGN6QWVGdzB5TXpFd01EVXlNRFF5TVRaYUZ3MHpNekV3TURJeU1EUXlNVFphTUJVeApFekFSQmdOVkJBTVRDbXQxWW1WeWJtVjBaWE13Z2dFaU1BMEdDU3FHU0liM0RRRUJBUVVBQTRJQkR3QXdnZ0VLCkFvSUJBUUN2cmtsWFNpQjludkdUVDZDRXBzMm85NUoyd2UydzFVZHNXSFI1ejBPS0tjN2YzM3ppWWRUdU5FT3IKMmxvQ0I0Y3NteDBWaWthOS9IRTZDQWpoaFkyeUVlcG4zMXh1eGl4VEYzY3Q5QzBsVGRFeFcraGVxU2JwdlExeAo5c0ZhOEZZZ3AxODZSUWh2enpQUEtPUHVtN1BmemFFK0pjVEh1V3pGYzlYcThnaE5qRVA3YzRZdTEyaDhuaWROCitvWWd3V0ljQldBMTBJWS9iNlZxVk9BVFZiejk3Q3JRWCtNNS9NaHg3am55WXBnVXBWM1pIVWJwMU8vRDQ1WWEKNXJjRTRDUGZHU0MxQTZkRU96Zjl6QnowT1B1RlN4R1lLTXJPNHE3dkpaUlZHRG8vbVRaOXo0SEVTcU13cFB0cAovTmU0ays3c1ZzNUxVRkZEcHlOWFNCbjI4blpaQWdNQkFBR2pXVEJYTUE0R0ExVWREd0VCL3dRRUF3SUNwREFQCkJnTlZIUk1CQWY4RUJUQURBUUgvTUIwR0ExVWREZ1FXQkJUSVFQa2JGRmFYdkxiKzMzVCtsMEhYSzM4cFB6QVYKQmdOVkhSRUVEakFNZ2dwcmRXSmxjbTVsZEdWek1BMEdDU3FHU0liM0RRRUJDd1VBQTRJQkFRQnZqZWxsMnozMgp2YXphVExNMzdoN2cyRHppUElEcUg4OVB4Y1d6NW91WGs1ZlA0QzBJcHI1MEtnRDRHN3RBRUFuelNGRWhEUHAyCjBUQ2ZnbkxsMktwT0RqekxydWxqR1dCWEhMR3ovS3RlZGV4QVhZMzZlOG5UNmxnSW55YzlNWEpROVUzYmNOMnIKVERXT2ptU2w5UXRwNEVDclVtMGlleUZCNDFLU01RSlNneWNZRE1JRjl4dDBnNy90MnBOZU1vcFBtUmdJM0NVZQo3VFF4L0Y4ZGZuSWxYNHJLUWY3TlR5Ulloek51QUY3cGFRWGQwWGQwcDZZRnM2aGtVRU1IRUxLMlFmQURFRS9aCjI2Tm8zakVkN2ZLdnF6T3YvQ3dzdEdXVTYzQUFPT3gyTTFCY28zNE9QRU95bTdmeXdBTWNlWlZUd0hzTVZnL2YKNm5pTWFPcGNSMEdwCi0tLS0tRU5EIENFUlRJRklDQVRFLS0tLS0K

%env cluster_endpoint=https://E79B2461776A9E2301CC3963E2FBF206.gr7.us-east-2.eks.amazonaws.com

%env cluster_name=AppD-EKS-Addon-umkht-EKS

%env token=k8s-aws-v1.aHR0cHM6Ly9zdHMuYW1hem9uYXdzLmNvbS8_QWN0aW9uPUdldENhbGxlcklkZW50aXR5JlZlcnNpb249MjAxMS0wNi0xNSZYLUFtei1BbGdvcml0aG09QVdTNC1ITUFDLVNIQTI1NiZYLUFtei1DcmVkZW50aWFsPUFLSUFSUUlLR01EV0tTUDdGQUZPJTJGMjAyMzEwMjklMkZ1cy1lYXN0LTElMkZzdHMlMkZhd3M0X3JlcXVlc3QmWC1BbXotRGF0ZT0yMDIzMTAyOVQyMjE2MjBaJlgtQW16LUV4cGlyZXM9MCZYLUFtei1TaWduZWRIZWFkZXJzPWhvc3QlM0J4LWs4cy1hd3MtaWQmWC1BbXotU2lnbmF0dXJlPWJlNDIyYWY1NTU3ZWE1YTE4YTQ0MzRkMWU3MzY5ZjA2ODcxNWI2Y2QwMGRhODJmMjgzMGE3OGVkNTM1Njc4YjA

%env aws_region=us-east-2

%env clientId=agt_1fbuVmNNxGnNJlv4ZAWyZD

%env clientSecret=hMSQ-pHoQwUpq7zbnzl66aqKUMge9kI_ZEX6hWkZWjU

%env clusterName=AppD-EKS-Addon-umkht-EKS

%env APPD_CLIENTID_POST=<provided in the Sandbox above>

%env APPD_SECRET_POST=<provided in the Sandbox above>

%env APPD_CLIENTID_BASIC=<provided in the Sandbox above>

%env APPD_SECRET_BASIC=<provided in the Sandbox above>

%env TENANT_NAME=<provided in the Sandbox above>

%env AWS_ACCESS_KEY=<provided in the Sandbox above>

%env AWS_SECRET_KEY=<provided in the Sandbox above>

%env AWS_CONNECTION_NAME=<provided in the Sandbox above>

%env APPD_CLIENTID_AGT=<provided in the Sandbox above>

%env APPD_SECRET_AGT=<provided in the Sandbox above>

## Deploy Teastore Multi service application in EKS


In [ ]:
%%writefile main.tf

provider "aws" {
  region = var.aws_region
}

provider "helm" {
  kubernetes {
    host                   = var.endpoint
    cluster_ca_certificate = base64decode(var.cluster_ca)
    token                  = var.token
  }
}

variable "endpoint" {
  type        = string
}
variable "cluster_ca" {
  type        = string
}
variable "token" {
  type        = string
}

resource "helm_release" "teaapp" {
  name       = "teaapp"
  description      = "A Helm chart for the TeaStore application."
  namespace = "default"
  chart = "https://prathjan.github.io/helm-chart/teastore-0.1.0.tgz"
  wait             = true
  wait_for_jobs    = true
}

In [ ]:
!terraform init
!terraform apply -auto-approve -var="cluster_ca=$cluster_ca" -var="endpoint=$cluster_endpoint" -var="token=$token"


## Access TeaStore Application

Get the service url for the TeaStore Application just deployed:


In [ ]:
!kubectl get pods --all-namespaces
 

In [ ]:
!kubectl get svc
%pwd


Use the Load Balancer IP listed above and access the application just deployed at:

http://*LB_IP*/tools.descartes.teastore.webui/

It should look something like this. Deployment takes about 5 mins, refresh if you do not see the UI below:

https://github.com/prathjan/images/blob/main/teaapp.png?raw=true

## Deploy AppDynamics Kubernetes and App Service Monitoring in EKS

Kubernetes and App Service Monitoring provides visibility into your Kubernetes® infrastructure and services for Application Performance Monitoring (APM).

For more info: 

https://docs.appdynamics.com/fso/cloud-native-app-obs/en/kubernetes-and-app-service-monitoring/install-kubernetes-and-app-service-monitoring#InstallKubernetesandAppServiceMonitoring-helm-chartsInstallKubernetesandAppServiceMonitoringUsingHelmCharts


### Pre-provisioning

Let's set up AppDynamics collector and operator configuration as per the steps in the above URL.

In [ ]:
%pwd

In [ ]:
%%writefile template.tf

variable "clusterName" {
  description = "AWS EKS Cluster name."
  type        = string
}

variable "clientId" {
  description = "FSO Platform client id."
  type        = string
}
variable "clientSecret" {
  description = "FSO Platform client secret."
  type        = string
}

locals {
    collector_content = templatefile("${path.module}/collectors-values.tmpl", { clusterName = var.clusterName, clientId = var.clientId, clientSecret = var.clientSecret })
    operator_content = templatefile("${path.module}/operators-values.tmpl", { clusterName = var.clusterName, clientId = var.clientId, clientSecret = var.clientSecret })
}

resource "null_resource" "local" {
  # Render to local file on machine
  # https://github.com/hashicorp/terraform/issues/8090#issuecomment-291823613
  provisioner "local-exec" {
    command = format(
      "cat <<\"EOF\" > \"%s\"\n%s\nEOF",
      "${path.module}/collectors-values.yaml",
      local.collector_content
    )
  }
  provisioner "local-exec" {
    command = format(
      "cat <<\"EOF\" > \"%s\"\n%s\nEOF",
      "${path.module}/operators-values.yaml",
      local.operator_content
    )
  }
}


resource "helm_release" "cert-manager" {
  name       = "cert-manager"
  description      = "A Helm chart for the cert-manager application."
  namespace = "cert-manager"
  create_namespace = true  
  chart            = "cert-manager"
  repository       = "https://charts.jetstack.io"
  version          = "v1.5.3"
  set {
    name  = "installCRDs"
    value = "true"
  }
}


In [ ]:
%pwd
!terraform init
!terraform apply -auto-approve -var="cluster_ca=$cluster_ca" -var="endpoint=$cluster_endpoint" -var="token=$token" -var="clusterName=$clusterName" -var="clientId=$clientId" -var="clientSecret=$clientSecret"


## Provision AppDynamics APM Monitoring

Let's deploy the collector and operator entities to start the monitoring of the EKS cluster.

In [ ]:
%%writefile appd.tf

terraform {
  required_providers {
    helm = {
      source = "hashicorp/helm"
    }
  }
}



resource helm_release appdoperator {
  name       = "appdynamics-operators"
  namespace = "appdynamics"
  create_namespace = true
  chart = "appdynamics-operators"
  repository = "https://appdynamics.jfrog.io/artifactory/appdynamics-cloud-helmcharts"
  timeout = 600
  values = [
    file("${path.module}/operators-values.yaml")
  ]
}

resource helm_release appdcollector {
  depends_on  = [helm_release.appdoperator]
  name       = "appdynamics-collectors"
  namespace = "appdynamics"
  chart = "appdynamics-collectors"
  repository = "https://appdynamics.jfrog.io/artifactory/appdynamics-cloud-helmcharts"
  timeout = 600
  values = [
    file("${path.module}/collectors-values.yaml")
  ]
}


In [ ]:
%pwd
!terraform init
!terraform apply -auto-approve -var="cluster_ca=$cluster_ca" -var="endpoint=$cluster_endpoint" -var="token=$token" -var="clusterName=$clusterName" -var="clientId=$clientId" -var="clientSecret=$clientSecret"


## Auto Instrumentation of TeaStore Application

Description - This section describes how to auto-instrument your services using the OpenTelemetry Operator For Kubernetes® installed by the Kubernetes and App Service Monitoring Helm charts above.


In [ ]:
%%writefile instrument.tf

resource "null_resource" "patchnamespace" {
   provisioner "local-exec" {
    command = <<EOT
      kubectl patch namespace default --patch '{"metadata": {"annotations": {"instrumentation.opentelemetry.io/inject-java": "appdynamics/appd-instrumentation"}}}'
  EOT
  }
}

resource "null_resource" "restartapp" {
  depends_on  = [null_resource.patchnamespace]
   provisioner "local-exec" {
    command = <<EOT
      kubectl delete pod $(kubectl get pods | grep teaapp | cut -d ' ' -f 1)
  EOT
  }
}



In [ ]:
%pwd
!terraform init
!terraform apply -auto-approve -var="cluster_ca=$cluster_ca" -var="endpoint=$cluster_endpoint" -var="token=$token" -var="clusterName=$clusterName" -var="clientId=$clientId" -var="clientSecret=$clientSecret"


## Full Stack Observability in AppDynamics Cloud

Once the JAVA services are auto instrumented, you will be able to review the value add of Cisco's FSO Platform as it applies to the following:

* Observe cloud-native microservices deployed on multiple public Kubernetes clusters.

* Observe the full stack, including relationships and interdependencies, using an extensible topology.

* Monitor business transactions.

* Receive MELT data from OpenTelemetry-compatible collectors.

https://github.com/prathjan/images/blob/main/fsosvc.png?raw=true


In the next section, we will review Cisco's FSO Platform briefly.